In [1]:
import pandas as pd
import statsmodels.api as sm

In [2]:
# Make sure your path is in seattle/scripts folder
seattle = pd.read_csv("FullSet_clean.csv")
seattle.columns 

Index(['Year', 'Month', 'Precinct', 'Sector', 'Beat', 'Total_calls',
       'Avg_response_time', 'No_action', 'Assistance', 'Citation',
       'Disturbance', 'Arrest', 'Domestic_violence', '911', 'Police_initiated',
       'Public_initiated', 'Alarm', 'ratio_domestic_to_all_calls',
       'ratio_citation_to_arrest', 'ratio_911_to_all_calls',
       'ratio_no_action_to_all_calls', 'ratio_public_to_police_initiated',
       'ratio_assistance_to_all_calls', 'Day', 'Date'],
      dtype='object')

In [3]:
# function to convert arima vals to dictionary
def setParam(North, West, East, South, SouthWest):
    out = {}
    out["NORTH"] = North
    out["WEST"] = West
    out["EAST"] = East
    out["SOUTH"] = South
    out["SOUTHWEST"] = SouthWest
    return out

In [4]:
# The main arima function 
def doArima(feature, precinct, params, df_forecast, df: seattle):
    dfAR = seattle[seattle["Precinct"] == precinct]
    ratios = [
        "ratio_public_to_police_initiated",
        "Avg_response_time",
        "ratio_domestic_to_all_calls",
        "ratio_no_action_to_all_calls",
    ]
    if feature in ratios:
        dfAR = dfAR[["Date", feature]].groupby("Date").mean(feature)
    else:
        dfAR = dfAR[["Date", feature]].groupby("Date").sum(feature)
    # Run Sarima
    drops = {0: 0, 1: 12, 2: 13}
    drop = drops[params[1]]
    mdat = dfAR[feature].dropna()[: (120 - drop)]
    model = sm.tsa.statespace.SARIMAX(mdat, order=params[0:3], seasonal_order=params)
    results = model.fit()
    # Add fcast to df_forecast
    fcast = results.get_forecast(drop + 10).summary_frame(alpha=0.05)
    # Keep only 2020 and last month 2019 and drop SE
    fcast = fcast.iloc[-11:, [0, 2, 3]]
    # Add new row so links with observed data better
    fcast.iloc[0, 0] = dfAR[dfAR.index == "2019-12-01"].values
    # Rename columns using feature/precinct
    fcast.columns = [
        f"{feature}_{precinct}_mean",
        f"{feature}_{precinct}_lb",
        f"{feature}_{precinct}_ub",
    ]
    df_forecast = df_forecast.join(fcast, how="left")
    # Drop any neg vals
    df_forecast = df_forecast.clip(lower=0)
    return df_forecast

In [5]:
# Precinct names
precincts = ["NORTH", "WEST", "EAST", "SOUTH", "SOUTHWEST"]

# Set up a dataframe to merge forecast data onto
dforecast = seattle[["Date", "Year"]]
dforecast = dforecast[dforecast["Date"] >= "2019-12-01"]
dforecast = dforecast[["Date", "Year"]].groupby("Date").max("Year")

# Input the data from the arima.xlsx here
fparams = {}
fparams["Total_calls"] = setParam(
    (2, 1, 4, 12), (2, 1, 7, 12), (2, 1, 7, 12), (2, 1, 3, 12), (0, 1, 4, 12)
)
fparams["Avg_response_time"] = setParam(
    (1, 1, 1, 12), (3, 1, 3, 12), (1, 1, 2, 12), (4, 1, 1, 12), (0, 1, 1, 12)
)
fparams["Arrest"] = setParam(
    (2, 0, 0, 12), (3, 0, 1, 12), (2, 0, 3, 12), (2, 1, 1, 12), (1, 1, 1, 12)
)
fparams["ratio_public_to_police_initiated"] = setParam(
    (1, 1, 2, 12), (0, 1, 6, 12), (0, 1, 3, 12), (1, 1, 1, 12), (2, 0, 1, 12)
)
fparams["ratio_domestic_to_all_calls"] = setParam(
    (1, 1, 1, 12), (2, 0, 1, 12), (1, 0, 0, 12), (0, 1, 0, 12), (0, 1, 0, 12)
)
fparams["ratio_no_action_to_all_calls"] = setParam(
    (2, 1, 0, 12), (1, 1, 0, 12), (0, 1, 1, 12), (0, 1, 0, 12), (2, 0, 0, 12)
)
fparams["Assistance"] = setParam(
    (0, 1, 2, 12), (1, 1, 0, 12), (2, 1, 3, 12), (3, 0, 4, 12), (1, 0, 4, 12)
)
fparams["Disturbance"] = setParam(
    (3, 0, 1, 12), (2, 1, 0, 12), (2, 1, 0, 12), (2, 0, 1, 12), (0, 1, 2, 12)
)
fparams["Citation"] = setParam(
    (2, 1, 3, 12), (2, 2, 1, 12), (2, 1, 1, 12), (2, 2, 1, 12), (2, 0, 1, 12)
)

In [6]:
for fname in fparams:
    print(f"Running ARIMA for feature {fname}")
    prec = fparams[fname]
    for pname in prec:
        print(f" and for precinct {pname}")
        #
        dforecast = doArima(
            feature=fname,
            precinct=pname,
            params=prec[pname],
            df_forecast=dforecast,
            df=seattle)


Running ARIMA for feature Total_calls
 and for precinct NORTH


C:\ProgramData\Miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  % freq, ValueWarning)
C:\ProgramData\Miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  % freq, ValueWarning)
C:\ProgramData\Miniconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:868: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  ' zeros.' % warning_description)
C:\ProgramData\Miniconda3\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


 and for precinct WEST


C:\ProgramData\Miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  % freq, ValueWarning)
C:\ProgramData\Miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  % freq, ValueWarning)
C:\ProgramData\Miniconda3\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


 and for precinct EAST


...

C:\ProgramData\Miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  % freq, ValueWarning)
C:\ProgramData\Miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  % freq, ValueWarning)


 and for precinct SOUTH


C:\ProgramData\Miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  % freq, ValueWarning)
C:\ProgramData\Miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  % freq, ValueWarning)


 and for precinct SOUTHWEST


C:\ProgramData\Miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  % freq, ValueWarning)
C:\ProgramData\Miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  % freq, ValueWarning)
C:\ProgramData\Miniconda3\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [7]:
dforecast.to_csv("ARIMA_forecast.csv", index=True)